# OpenAI Whisper 语音识别示例

## 1. 安装Whisper

In [ ]:
# 在单元格中运行安装命令
!pip install -U openai-whisper
# 如果需要GPU加速（推荐NVIDIA显卡）
!pip install git+https://github.com/openai/whisper.git

## 2. 基本使用示例

In [ ]:
import whisper

# 加载模型（首次运行会自动下载）
# 可选模型：tiny, base, small, medium, large
model = whisper.load_model("base")

# 转录音频文件
result = model.transcribe("audio.mp3")
print(result["text"])

## 3. 高级选项示例

In [ ]:
import whisper

model = whisper.load_model("medium")

# 带参数配置的转录
result = model.transcribe(
    "audio.wav",
    language="zh",  # 指定语言（中文）
    fp16=False,     # 如果使用CPU需要设为False
    temperature=0.2  # 控制随机性（0-1）
)

# 输出结果
print("文本:", result["text"])
print("分段信息:")
for segment in result["segments"]:
    print(f"[{segment['start']:.1f}s -> {segment['end']:.1f}s] {segment['text']}")

## 4. 实时麦克风输入（需要pyaudio）

In [ ]:
import whisper
import pyaudio
import wave
import numpy as np

# 录音参数
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5

p = pyaudio.PyAudio()

# 开始录音
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("开始录音...")
frames = []
for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(np.frombuffer(data, dtype=np.int16))

# 停止录音
stream.stop_stream()
stream.close()
p.terminate()

# 转换为Whisper需要的格式
audio = np.concatenate(frames).astype(np.float32) / 32768.0

# 加载模型并转录
model = whisper.load_model("base")
result = model.transcribe(audio, language="zh")
print(result["text"])

## 5. 翻译为英语（仅限medium/large模型）

In [ ]:
model = whisper.load_model("large")
result = model.transcribe("audio.wav", task="translate")  # 翻译为英文
print(result["text"])

## 注意事项
1. 模型越大精度越高，但速度越慢（base适合英语，中文建议至少small）
2. 首次运行会自动下载模型（保存在 ~/.cache/whisper）
3. 处理长音频可能需要大量内存

如需处理视频文件，可以先用ffmpeg提取音频：
```bash
ffmpeg -i video.mp4 -vn -acodec copy audio.m4a
```